In [1]:
import pandas as pd
import numpy as np
from io import StringIO
%matplotlib inline
from rdflib import Graph,URIRef, tools
from gastrodon import LocalEndpoint,one,QName
pd.set_option("display.width",100)
pd.set_option("display.max_colwidth",80)
import statistics
from csvwlib import CSVWConverter
import rdflib

## Load data 

In [ ]:
input_path = "/media/rivas/Data1/Data-mining/Kcap/datasets/Yago10/"

y10_type = input_path +"Yago10_Type/" + "Yago10_Type.nt"
#y_s_taxonomy = input_path + "yagoSimpleTaxonomy.ttl"
y_s_taxonomy = input_path + "yagoTaxonomy.ttl"

yago_taxonomy = Graph()
yago_taxonomy.parse(y_s_taxonomy, format="turtle")
#yago_type = Graph()
#yago_type.parse(y_s_type, format="turtle")
#drugbank.parse(f_drugbank, format="turtle")
yago10_type = Graph()
yago10_type.parse(y10_type, format="nt")

print(len(yago10_type))
print(len(yago_taxonomy))

## Create local endpoint (yago_type and yago_taxonomy)

In [37]:
print(len(yago10_type))
print(len(yago_taxonomy))
y_type=LocalEndpoint(yago10_type)
y_taxonomy=LocalEndpoint(yago_taxonomy)

1587343
1754641


## Select all relation with person type

In [38]:
"""
   SELECT distinct ?s ?p WHERE{
      ?s ?p <http://yago-knowledge.org/resource/wordnet_person_100007846>.
   }
"""

spo_person = y_type.select("""
   SELECT distinct ?s ?p ?o WHERE{
      ?s ?p <http://yago-knowledge.org/resource/wordnet_person_100007846>.
      ?s ?p ?o 
   }
""")
obj_person = y_type.select("""
   SELECT distinct ?o WHERE{
      ?s ?p <http://yago-knowledge.org/resource/wordnet_person_100007846>.
      ?s ?p ?o 
   }
""")

In [39]:
spo_person.shape

(249263, 3)

In [40]:
obj_person.head()

,o
0,http://yago-knowledge.org/resource/wordnet_person_100007846
1,http://yago-knowledge.org/resource/wikicat_African-American_people
2,http://yago-knowledge.org/resource/wikicat_American_record_producers
3,http://yago-knowledge.org/resource/wikicat_American_screenwriters
4,http://yago-knowledge.org/resource/wikicat_Musical_theatre_actors


In [41]:
p = y_taxonomy.select("SELECT distinct ?p WHERE{ ?s ?p ?o}")
p.head()

,p
0,rdfs:subClassOf
1,http://yago-knowledge.org/resource/hasGloss


## Select taxonomy of all relation with person type

In [ ]:
taxonomy = pd.DataFrame()
for t in range(obj_person.shape[0]):
    obj = '<'+ obj_person['o'][t] +'>'
    query = 'SELECT distinct ?p ?o WHERE{ \n' +obj+ '?p ?o \n}'
    pred_obj_person = y_taxonomy.select(query)
    pred_obj_person.insert(0, "s", obj)
    frames = [taxonomy, pred_obj_person]
    taxonomy = pd.concat(frames)

In [43]:
taxonomy.head()

,s,p,o
0,<http://yago-knowledge.org/resource/wordnet_person_100007846>,rdfs:subClassOf,http://yago-knowledge.org/resource/yagoLegalActor
1,<http://yago-knowledge.org/resource/wordnet_person_100007846>,rdfs:subClassOf,http://yago-knowledge.org/resource/wordnet_causal_agent_100007347
2,<http://yago-knowledge.org/resource/wordnet_person_100007846>,rdfs:subClassOf,http://yago-knowledge.org/resource/wordnet_organism_100004475
0,<http://yago-knowledge.org/resource/wikicat_African-American_people>,rdfs:subClassOf,http://yago-knowledge.org/resource/wordnet_person_100007846
1,<http://yago-knowledge.org/resource/wikicat_African-American_people>,rdfs:subClassOf,http://yago-knowledge.org/resource/wikicat_American_people_of_African_descent


In [44]:
pred_obj_person = y_taxonomy.select("""
   SELECT distinct ?p ?o WHERE{
      <http://yago-knowledge.org/resource/wordnet_person_100007846> ?p ?o
   }
""")

In [45]:
pred_obj_person.head()

,p,o
0,rdfs:subClassOf,http://yago-knowledge.org/resource/yagoLegalActor
1,rdfs:subClassOf,http://yago-knowledge.org/resource/wordnet_causal_agent_100007347
2,rdfs:subClassOf,http://yago-knowledge.org/resource/wordnet_organism_100004475


## Preapre dataframe to save in .nt 

In [ ]:
pred_obj_person.insert(0, "s", '<http://yago-knowledge.org/resource/wordnet_person_100007846>') 
pred_obj_person['p'] = pred_obj_person['p'].str.replace('owl:disjointWith', '<http://www.w3.org/2002/07/owl#disjointWith>')
pred_obj_person['p'] = pred_obj_person['p'].str.replace('rdfs:subClassOf', '<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>')
pred_obj_person['o'] = '<' + pred_obj_person['o'] + '> .'
pred_obj_person['o'] = pred_obj_person['o'].str.replace('<owl:Thing> .', '<http://www.w3.org/2002/07/owl#owl:Thing> .')

taxonomy['p'] = taxonomy['p'].str.replace('owl:disjointWith', '<http://www.w3.org/2002/07/owl#disjointWith>')
taxonomy['p'] = taxonomy['p'].str.replace('rdfs:subClassOf', '<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>')
taxonomy['o'] = '<' + taxonomy['o'] + '> .'
taxonomy['o'] = taxonomy['o'].str.replace('<owl:Thing> .', '<http://www.w3.org/2002/07/owl#owl:Thing> .')

frames = [pred_obj_person, taxonomy]
yago10_tax = pd.concat(frames)

#pred_obj_person.to_csv(input_path + 'Yago10_taxonomy_person.nt', sep=' ', index = None, header=False)
yago10_tax.to_csv(input_path + 'Yago10_taxonomy_person.nt', sep='\t', index = None, header=False)

In [47]:
yago10_tax.head()

,s,p,o
0,<http://yago-knowledge.org/resource/wordnet_person_100007846>,<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>,<http://yago-knowledge.org/resource/yagoLegalActor> .
1,<http://yago-knowledge.org/resource/wordnet_person_100007846>,<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>,<http://yago-knowledge.org/resource/wordnet_causal_agent_100007347> .
2,<http://yago-knowledge.org/resource/wordnet_person_100007846>,<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>,<http://yago-knowledge.org/resource/wordnet_organism_100004475> .
0,<http://yago-knowledge.org/resource/wordnet_person_100007846>,<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>,<http://yago-knowledge.org/resource/yagoLegalActor> .
1,<http://yago-knowledge.org/resource/wordnet_person_100007846>,<http://www.w3.org/2000/01/rdf-schema#rdfs:subClassOf>,<http://yago-knowledge.org/resource/wordnet_causal_agent_100007347> .


In [48]:
taxonomy.shape

(79787, 3)

In [49]:
yago10_tax.shape

(79790, 3)

In [50]:
pred_obj_person.shape

(3, 3)

## Filter yago10 dataset by person

In [51]:
dataset_yago = Graph()
dataset_yago.parse(input_path + 'yago10.nt', format="nt")
dataset_type = Graph()
dataset_type.parse(input_path + 'Yago10_type_person.nt', format="nt")

<Graph identifier=N453a7ef82f354f98b3425cfb9e004525 (<class 'rdflib.graph.Graph'>)>

In [52]:
print(len(dataset_yago))
print(len(dataset_type))
dataset_y=LocalEndpoint(dataset_yago)
dataset_type_person=LocalEndpoint(dataset_type)

1089040
249263


In [53]:
subject = dataset_type_person.select("""
   SELECT distinct ?s WHERE{
      ?s ?p ?o
   }
""")

In [54]:
subject.shape

(12077, 1)

In [55]:
subject['s'][12076]

rdflib.term.URIRef('http://yago-knowledge.org/resource/Josef_Armberger')

## Select only the triples related to person

In [ ]:
yago10 = pd.DataFrame()
for t in range(subject.shape[0]):
    sub = '<' + subject['s'][t] + '>'
    query = 'SELECT distinct ?p ?o WHERE{ \n' +sub+ ' ?p ?o \n}'
    pred_obj_person = dataset_y.select(query)
    pred_obj_person.insert(0, "s", sub)
    pred_obj_person['p'] = '<' + pred_obj_person['p'] +'>'
    pred_obj_person['o'] = '<' + pred_obj_person['o'] +'> .'
    frames = [yago10, pred_obj_person]
    yago10 = pd.concat(frames)

In [ ]:
"""
yago10_person = pd.DataFrame()
for sub in range(datset_yago10_person.shape[0]):
    subject = datset_yago10[datset_yago10[0].isin({datset_yago10_person[0][sub]})]
    frames = [yago10_person, subject]
    yago10_person = pd.concat(frames)
"""

In [57]:
print(yago10.shape)

(101987, 3)


In [58]:
yago10.head()

,s,p,o
0,<http://yago-knowledge.org/resource/Irene_Worth>,<http://yago-knowledge.org/resource/hasWonPrize>,<http://yago-knowledge.org/resource/Drama_Desk_Award> .
1,<http://yago-knowledge.org/resource/Irene_Worth>,<http://yago-knowledge.org/resource/diedIn>,<http://yago-knowledge.org/resource/New_York_City> .
2,<http://yago-knowledge.org/resource/Irene_Worth>,<http://yago-knowledge.org/resource/hasWonPrize>,<http://yago-knowledge.org/resource/Emmy_Award> .
3,<http://yago-knowledge.org/resource/Irene_Worth>,<http://yago-knowledge.org/resource/hasWonPrize>,<http://yago-knowledge.org/resource/Tony_Award> .
4,<http://yago-knowledge.org/resource/Irene_Worth>,<http://yago-knowledge.org/resource/hasGender>,<http://yago-knowledge.org/resource/female> .


In [59]:
yago10 = yago10.drop_duplicates()
yago10.shape

(101987, 3)

In [60]:
yago10.to_csv(input_path + 'Yago10_person/without semantics/' + 'yago_dataset_person.nt', sep='\t', index=False, header=None, encoding='utf-8')